# Training the Schlager Bot Language Model

To train the model that will ultimately generate Schlager lyrics, we first train a BERT model from scratch on our text dataset composed of Schlager lyrics and some generic German text from an open source NLP training dataset. The latter serves to provide grammatical structure and vocabulary that the lyrics alone lack.

Pre-training on transformers can be done with self-supervised tasks. In this case we will use Masked Language Modeling (MLM), where a certain percentage of the tokens in a sentence is masked and the model is trained to predict those masked words. One of the advantages of this method is that it can see the position information of the whole sentence - both for the masked and visible part.

First we need to import the relevant dependencies.

In [21]:
import transformers
import json
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
import datasets
import os
from tokenizers import BertWordPieceTokenizer
from tokenizers import BertTokenizerFast

ImportError: cannot import name 'BertTokenizerFast' from 'tokenizers' (/home/niclaswiegleb/miniconda3/envs/schlager_bot/lib/python3.11/site-packages/tokenizers/__init__.py)

Before we can start putting our model together, we need to preprocess the training data and split it into training and test sub-sets. Separating these out first avoids the model seeing the test data during training and risking overfitting.

In [9]:
#Loading the lyrics data as the dataset

#Iterating through the files in the directory and adding the names to the files variable

files = []

directory = "./lyrics"

for filename in os.scandir(directory):
    if filename.is_file():
        files.append(filename.path)

dataset = datasets.load_dataset("text", data_files= files, split= "train")

Resolving data files:   0%|          | 0/868 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /home/niclaswiegleb/.cache/huggingface/datasets/text/default-e8b327e4ca6a9b04/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


In [10]:
#Now we split the dataset into training (90%) and testing (10%)

d = dataset.train_test_split(test_size= 0.1)
d["train"], d["test"]

(Dataset({
     features: ['text'],
     num_rows: 34607
 }),
 Dataset({
     features: ['text'],
     num_rows: 3846
 }))

Now that the dataset is loaded and split into training and test data, it is time to train the tokenizer. To achieve this, we need to write our dataset into text files - keeping training and test data separate.

In [11]:
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer 
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset["text"]:
      print(t, file=f)

# save the training set to train.txt
dataset_to_text(d["train"], "./model/model_data/train.txt")
# save the testing set to test.txt
dataset_to_text(d["test"], "./model/model_data/test.txt")

Next we define some parameters of the tokenizer. The training file indicates the data we're passing to the tokenizer for training. This could be a list of files too. vocab_size is the vocabulary size of tokens, while max_length is the maximum sequence length.

truncate_longer_samples is a boolean indicating whether we truncate sentences longer than the length of max_length, if it's set to False, we won't truncate the sentences, we group them together and split them by max_length, so all the resulting sentences will have the length of max_length.

In [19]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]

training_file = ["./model/model_data/train.txt"]
# 30,522 vocab is BERT's default vocab size
vocab_size = 30_522
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = False

We are now ready to train the tokenizer. BERT's default tokenizer is WordPiece and, therefore, we initialize the BertWordPieceTokenizer() tokenizer class from the tokenizers library and use the train() method to train it. It will take several minutes to finish. We then need to save the tokenizer to a directory.

In [20]:
#Initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()

#Train the tokenizer
tokenizer.train(files=training_file, vocab_size=vocab_size, special_tokens=special_tokens)

# #Enable truncation up to the maximum 512 tokens
# tokenizer.enable_truncation(max_length=max_length)

model_path = "./model/tokenizer/"

#Save the tokenizer to directory
tokenizer.save_model(model_path)

#Dumping some of the tokenizer config to config file, 
#including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

The tokenizer.save_model() method saves the vocabulary file into that path, we also manually save some tokenizer configurations, such as special tokens:

- unk_token: A special token that represents an out-of-vocabulary token, even though the tokenizer is a WordPiece tokenizer, the unk tokens are not
- impossible, but rare.
- sep_token: A special token that separates two different sentences in the same input.
- pad_token: A special token that is used to fill sentences that do not reach the maximum sequence length (since the arrays of tokens must be the same size).
- cls_token: A special token representing the class of the input.
- mask_token: This is the mask token we use for the Masked Language Modeling (MLM) pretraining task.

In [22]:
#Let's load the tokenizer
tokenizer = transformers.BertTokenizerFast.from_pretrained(model_path)

With the tokenizer ready to be taken into operation, we can now tokenize our dataset.

In [24]:
#Tokenizing the training dataset
train_dataset = d["train"].map((lambda x: tokenizer(x["text"], return_special_tokens_mask=True)), batched= True)

#Tokenizing the test dataset
test_dataset = d["train"].map((lambda x: tokenizer(x["text"], return_special_tokens_mask=True)), batched= True)



Map:   0%|          | 0/34607 [00:00<?, ? examples/s]